# why flox?

```
/////
File: why_flox
Author: Thomas Moore
Description: This notebook explores examples showing the benefit of `flox` and will run on a laptop
Date: 1 May 2024
/////
```

In [1]:
Author_dict = {"name": "Thomas Moore", 
               "affiliation": "CSIRO", 
               "email": "thomas.moore@csiro.au",
               "orchid_ID":'https://orcid.org/0000-0003-3930-1946'}

# setup

In [2]:
import dask
import flox
import xarray as xr
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster

In [3]:
def print_chunks(data_array):
    chunks = data_array.chunks
    dim_names = data_array.dims
    readable_chunks = {dim: chunks[i] for i, dim in enumerate(dim_names)}
    for dim, sizes in readable_chunks.items():
        print(f"{dim} chunks: {sizes}")

In [4]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def plot_temperature_at_time(ds, time_index, depth_index=0):
    # Select the data for a specific time and depth
    temp_data = ds['temperature'].isel(time=time_index, depth=depth_index)
    
    # Create a plot
    fig = plt.figure(figsize=(15, 7))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_global()
    
    # Add map features
    ax.add_feature(cfeature.LAND, zorder=100, edgecolor='k')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Plot the temperature data
    temp_plot = ax.pcolormesh(ds.longitude, ds.latitude, temp_data, transform=ccrs.PlateCarree(), cmap='viridis')

    # Add a color bar
    cbar = plt.colorbar(temp_plot, orientation='vertical', pad=0.02, aspect=50)
    cbar.set_label('Temperature (degrees Celsius)')

    # Add title
    plt.title(f'Ocean Temperature on {ds.time[time_index].values}')
    plt.show()


# start LocalCluster

In [5]:
cluster = LocalCluster(
    n_workers=4,          # Number of workers
    threads_per_worker=1 # Number of threads per each worker
)
client = Client(cluster)

2024-05-02 10:15:18,873 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,875 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,885 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,900 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,983 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,985 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,993 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:18,996 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:19,296 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:19,298 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:19,305 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:19,315 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:20,321 - distributed.nanny - WARNING - Restarting worker
2024-05-02 10:15:20,323 - distributed.

# make example xarray object & write to `netcdf`

In [ ]:
# Define dimensions
n_time = 33 * 365  # approximately, not accounting for leap years
n_leap = 8 # manually calculated for the period 1990 + 33 years
n_time = n_time + n_leap
n_lat = 60
n_lon = 144
n_depth = 20

In [ ]:
# Create a time range from 1990-01-01 over 33 years
time = pd.date_range('1990-01-01', periods=n_time, freq='D')
time

In [ ]:
# Create latitude and longitude arrays
latitude = np.linspace(-90, 90, n_lat)  # example range from -90 to 90
longitude = np.linspace(0, 360, n_lon, endpoint=False)  # example range from 0 to 360
depth = np.linspace(0, 5000, n_depth)  # example range from 0 to 5000 meters

In [ ]:
# Initialize random temperature data
a = 1.0
b = 32.0
data = (a + (np.random.rand(n_time, n_lat, n_lon, n_depth) * (b - a))).astype(np.float32)

In [ ]:
# Apply depth scaling: Temperature decreases with depth
# Assuming a linear decrease with depth, adjust scale factor as needed
depth_scale_factor = np.linspace(1, 0.2, n_depth).astype(np.float32)  # From 100% at the surface to 20% at the maximum depth
depth_scaled_data = data * depth_scale_factor[np.newaxis, np.newaxis, np.newaxis, :]

In [ ]:
# Apply an exponential function to enhance the latitude effect
exponential_factor = np.exp(0.1 * np.abs(latitude)).astype(np.float32)  # Adjust the coefficient as needed to increase strength

# Inverse the exponential factor to decrease temperature towards the poles
latitude_scale_factor = 1 / exponential_factor.astype(np.float32)

# Reshape latitude_scale_factor for broadcasting
latitude_scale_factor = latitude_scale_factor[:, np.newaxis, np.newaxis].astype(np.float32)

# Apply latitude scaling
latitude_scaled_data = depth_scaled_data * (latitude_scale_factor + 1) / 2  # Adjust scale to range [0.5, 1]

In [ ]:
latitude_scaled_data = latitude_scaled_data.astype(np.float32)

In [ ]:
# Create the dataset
ds = xr.Dataset(
    {
        "temperature": (("time", "latitude", "longitude", "depth"),latitude_scaled_data)
    },
    coords={
        "time": time,
        "latitude": latitude,
        "longitude": longitude,
        "depth": depth
    }
)

# Assign attributes
ds['temperature'].attrs['units'] = 'degrees Celsius'
ds['temperature'].attrs['description'] = 'Simulated ocean temperatures'
#ds['time'].attrs['calendar'] = 'gregorian'


In [ ]:
ds.nbytes/1e9

In [ ]:
plot_temperature_at_time(ds, 0, 0)

In [ ]:
ds

In [ ]:
# Define chunking strategy
chunks = {
    'time': 100,    # Chunk per day
    'latitude': 60,  # Chunk latitude if necessary, depends on available memory
    'longitude': 144,  # Chunk longitude if necessary, can adjust based on use case
    'depth': 20    # Chunk by all depth levels if not too large
}

# Write the dataset to NetCDF
ds.chunk(chunks).to_netcdf('/Users/moo270/data/climatology-demo/dummy_ocean_temperature.nc', mode='w', format='NETCDF4', engine='netcdf4',
                           encoding={'temperature': {'chunksizes': (100, 60, 144, 20)}})

# RELOAD dataset from `netcdf`

In [ ]:
loaded_ds = xr.open_dataset('/Users/moo270/data/climatology-demo/dummy_ocean_temperature.nc',chunks={})

In [ ]:
loaded_ds

In [ ]:
ds_chunk_time_100 = loaded_ds

In [ ]:
ds_chunk_time_100

In [ ]:
# Retrieve Xarray options
options = xr.get_options()

# Convert the options dictionary to a Pandas DataFrame for a nicer table display
options_df = pd.DataFrame(list(options.items()), columns=['Option', 'Value'])

# Print the DataFrame
print(options_df)

In [ ]:
%%time
clim_flox = ds_chunk_time_100.groupby('time.month').mean('time').compute()

In [ ]:
clim_flox.nbytes/1e9

In [ ]:
%%time
with xr.set_options(use_flox=False):
    # Retrieve Xarray options
    options = xr.get_options()

    # Convert the options dictionary to a Pandas DataFrame for a nicer table display
    options_df = pd.DataFrame(list(options.items()), columns=['Option', 'Value'])

    # Print the DataFrame
    print(options_df)
    #
    clim_noflox = ds_chunk_time_100.groupby('time.month').mean('time').compute()

In [ ]:
import gc
#del clim_noflox
#del clim_flox
gc.collect()
client.restart()

# flox docs examples

In [13]:
import dask.array as da
# Generate a DataArray with random numbers
oisst = xr.DataArray(
    da.random.random((14532, 720, 144), chunks=(20, 720, 144)),  # Generate random values
    dims=("time", "lat", "lon"),
    coords={"time": pd.date_range("1981-09-01 12:00", "2021-06-14 12:00", freq="D")},
    name="sst"
)

In [14]:
oisst.nbytes/1e9

12.05342208

In [15]:
oisst.groupby('time.month').mean('time')

<xarray.DataArray 'sst' (month: 12, lat: 720, lon: 144)> Size: 10MB
dask.array<transpose, shape=(12, 720, 144), dtype=float64, chunksize=(1, 720, 144), chunktype=numpy.ndarray>
Coordinates:
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lat, lon

In [16]:
%%time
oisst.groupby('time.month').mean('time').compute()

CPU times: user 2.33 s, sys: 360 ms, total: 2.69 s
Wall time: 9.76 s


<xarray.DataArray 'sst' (month: 12, lat: 720, lon: 144)> Size: 10MB
array([[[0.5052987 , 0.4855434 , 0.50219747, ..., 0.50684906,
         0.49731357, 0.50095427],
        [0.48971857, 0.5017485 , 0.50336188, ..., 0.50440979,
         0.49505648, 0.51482027],
        [0.49112897, 0.50605026, 0.48869142, ..., 0.49922243,
         0.49300102, 0.50208827],
        ...,
        [0.49997539, 0.49497355, 0.48525562, ..., 0.51338945,
         0.49245455, 0.51355042],
        [0.494952  , 0.49032254, 0.51209663, ..., 0.49738907,
         0.49464667, 0.4923154 ],
        [0.498065  , 0.50088461, 0.50705006, ..., 0.4986839 ,
         0.50792527, 0.49777132]],

       [[0.50259873, 0.4848876 , 0.49955449, ..., 0.48781565,
         0.51539575, 0.50277657],
        [0.50389133, 0.5044048 , 0.49829216, ..., 0.51067253,
         0.491989  , 0.4826077 ],
        [0.5097113 , 0.51111381, 0.49720787, ..., 0.49366673,
         0.50159234, 0.48301122],
...
        [0.49075477, 0.50989046, 0.50009373, ..., 0.49095778,
         0.50562565, 0.51620227],
        [0.51009843, 0.49842435, 0.49646318, ..., 0.50909644,
         0.48700213, 0.51267728],
        [0.49426365, 0.50793737, 0.51134595, ..., 0.49046375,
         0.51173371, 0.49541739]],

       [[0.50324616, 0.50540132, 0.50647698, ..., 0.49825108,
         0.5010606 , 0.49913319],
        [0.50123384, 0.48888259, 0.50315302, ..., 0.49454797,
         0.51016925, 0.48585921],
        [0.501239  , 0.50311189, 0.49678798, ..., 0.50871837,
         0.50382289, 0.48570228],
        ...,
        [0.51594729, 0.50146948, 0.49981631, ..., 0.49691374,
         0.5071391 , 0.49983064],
        [0.48306225, 0.49970246, 0.48184749, ..., 0.49655806,
         0.50090503, 0.50866127],
        [0.49923644, 0.49426431, 0.5044125 , ..., 0.48862154,
         0.49752522, 0.49351398]]])
Coordinates:
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lat, lon

In [17]:
%%time
with xr.set_options(use_flox=False):
    # Retrieve Xarray options
    options = xr.get_options()

    # Convert the options dictionary to a Pandas DataFrame for a nicer table display
    options_df = pd.DataFrame(list(options.items()), columns=['Option', 'Value'])

    # Print the DataFrame
    print(options_df)
    #
    oisst.groupby('time.month').mean('time').compute()

                      Option    Value
0       arithmetic_broadcast     True
1            arithmetic_join    inner
2             cmap_divergent   RdBu_r
3            cmap_sequential  viridis
4           display_max_rows       12
5   display_values_threshold      200
6              display_style     html
7              display_width       80
8       display_expand_attrs  default
9      display_expand_coords  default
10  display_expand_data_vars  default
11       display_expand_data  default
12     display_expand_groups  default
13    display_expand_indexes  default
14   display_default_indexes    False
15        enable_cftimeindex     True
16        file_cache_maxsize      128
17                keep_attrs  default
18   warn_for_unclosed_files    False
19            use_bottleneck     True
20                  use_flox    False
21               use_numbagg     True
22            use_opt_einsum     True
CPU times: user 1.59 s, sys: 238 ms, total: 1.83 s
Wall time: 4.21 s
